In [1]:
import torch
from torchinfo import summary
import s3prl.hub as hub

Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



In [2]:
model = getattr(hub, "mockingjay")()  # build the Mockingjay model with pre-trained weights
print(model)

Using cache found in /home/chihyuan/.cache/torch/hub/s3prl_cache/a2b432be9adba2cb59f5cf89a4cf84d5fff8ec3c9fe248ad53349694565ef8c9
for https://www.dropbox.com/s/zwsfa6w2iy2cc68/states-500000.ckpt?dl=0
[UpstreamExpert] - Using the default upstream expert config
UpstreamExpert(
  (transformer): PretrainedTransformer(
    (extracter): OnlinePreprocessor(
      (_melscale): MelScale()
      (_mfcc_trans): MFCC(
        (amplitude_to_DB): AmplitudeToDB()
        (MelSpectrogram): MelSpectrogram(
          (spectrogram): Spectrogram()
          (mel_scale): MelScale()
        )
      )
    )
    (model): TransformerModel(
      (input_representations): TransformerInputRepresentations(
        (spec_transform): Linear(in_features=80, out_features=768, bias=True)
        (LayerNorm): TransformerLayerNorm()
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): TransformerEncoder(
        (layer): ModuleList(
          (0): TransformerLayer(
            (attention): Transforme

In [10]:
print(type(model.transformer))

<class 's3prl.upstream.mockingjay.builder.PretrainedTransformer'>


In [1]:
from s3prl import adapters

In [2]:
dir(adapters)

['ADAPTERFUSION_CONFIG_MAP',
 'ADAPTER_CACHE',
 'ADAPTER_CONFIG_MAP',
 'ADAPTER_MODEL_MAPPING',
 'AdapterArguments',
 'AdapterCompositionBlock',
 'AdapterConfig',
 'AdapterConfigBase',
 'AdapterFusionConfig',
 'AdapterInfo',
 'AdapterLayer',
 'AdapterLayerBase',
 'AdapterSetup',
 'AdapterType',
 'AutoAdapterModel',
 'AutoModelWithHeads',
 'BartAdapterModel',
 'BartModelWithHeads',
 'BatchSplit',
 'BertAdapterModel',
 'BertModelWithHeads',
 'CompacterConfig',
 'CompacterPlusPlusConfig',
 'ConfigUnion',
 'DEFAULT_ADAPTERFUSION_CONFIG',
 'DEFAULT_ADAPTER_CONFIG',
 'DistilBertAdapterModel',
 'DistilBertModelWithHeads',
 'DynamicAdapterFusionConfig',
 'ElectraAdapterModel',
 'ElectraModelWithHeads',
 'ForwardContext',
 'Fuse',
 'GPT2AdapterModel',
 'GPT2ModelWithHeads',
 'HoulsbyConfig',
 'HoulsbyInvConfig',
 'InvertibleAdaptersMixin',
 'MAMConfig',
 'MBartAdapterModel',
 'MBartModelWithHeads',
 'MODEL_WITH_HEADS_MAPPING',
 'ModelAdaptersConfig',
 'ModelAdaptersMixin',
 'ModelWithHeadsAdapt